# Projeto Banco de Dados I
### Professor Rogério Mainardes
### Alunos Erick Henrique e Paulo Vitor 

### Definição do projeto

Esse projeto consiste na análise descritiva de um dataset e de 10 perguntas cujas respostas serão dadas através de consultas usando a linguagem SQL.
O dataset escolhido está disponível no [github](https://github.com/ValdisW/datasets/blob/master/video-game-sales.csv).

### Descrição do dataset

Os dados escolhidos mostram as vendas de jogos de videogame lançados entre 1980 e 2020 que tiveram mais de 100.000 vendas nas regiões América do Norte, União Europeia, Japão ou outros (resto do mundo). Na tabela, as colunas *_sales mostram as vendas em milhões de unidades.

### Criação da tabela que será consultada no postgres

In [1]:
-- Criação da tabela videogame_sales no postgres
CREATE TABLE videogame_sales(
    ranking integer,
    game_name varchar(200),
    platform varchar(20),
    release_year smallint,
    genre varchar(20),
    publisher varchar(50),
    na_sales real,
    eu_sales real,
    jp_sales real,
    other_sales real,
    global_sales real
);

In [ ]:
-- cópia dos dados do arquivo videogame-sales para o postgres
COPY videogame_sales 
FROM 'caminho/do/arquivo/arquivo.csv'
WITH(
    FORMAT csv,
    DELIMITER ',', 
    HEADER,
    NULL 'N/A'
);

### Conectando com o banco de dados

In [ ]:
# Instalando libs necessárias
!pip install sqlalchemy
!pip install pandas
!pip install pandasql

In [2]:
# Importa as libs instaladas

from sqlalchemy import create_engine
import pandas as pd
import pandasql as ps

In [3]:
# Configura conexão com o banco

SGBD = "postgresql+psycopg2"
USER = "postgres"
SENHA = "123"
HOST = "localhost"
DATABASE = "Video game dataset"

engine = create_engine(f"{SGBD}://{USER}:{SENHA}@{HOST}/{DATABASE}")
conn = engine.connect()

In [4]:
# Consulta teste
pd.read_sql_query(
    """
        SELECT 
            * 
        FROM 
            videogame_sales
        LIMIT 4
    """, 
    con=conn
)

,ranking,game_name,platform,release_year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00


### Análise Descritiva

#### Tipagem das colunas

In [62]:
pd.read_sql_query(
    """
        SELECT 
            column_name, data_type
        FROM 
            information_schema.columns
        WHERE
            table_schema = 'public' AND table_name = 'videogame_sales'
    """, 
    con=conn
)

,column_name,data_type
0,ranking,integer
1,game_name,character varying
2,platform,character varying
3,release_year,smallint
4,genre,character varying
5,publisher,character varying
6,na_sales,real
7,eu_sales,real
8,jp_sales,real
9,other_sales,real


Information schema é um schema criado automaticamente pelo postgres e que contém informações de todas as colunas no banco de dados.<br>
Filtramos esse schema para mostrar apenas dados da tabela que criamos (videogame_sales) e, especificamente, as colunas column_name e data_type.<br>
Para verificar outras informações presentes nesse schema, altere o 'SELECT' para '*'.<br><br>
Tipos de dados presentes:
- integer e smallint: variáveis do tipo inteiro. integer é a escolha mais comum. Usamos smallint para representar os anos para usar um tipo diferente e por sabermos que não haveria números com mais de 4 dígitos nessa coluna
- character varying: representa variáveis do tipo texto que tem tamanho variável, tendo apenas um limite superior. Definimos esse limite como 50.
- real: tipo de dado numérico de ponto flutuante (o popular *float*). O real tem um range de 1E-37 a 1E+37 com uma precisão de pelomenos 6 casas decimais, sendo mais que o suficiente para os nossos propósitos.

#### Quantidade de valores nulos em cada coluna

In [71]:
pd.read_sql_query(
    """
        SELECT
            COUNT(*) FILTER (WHERE ranking IS NULL) AS nulos_ranking,
            COUNT(*) FILTER (WHERE game_name IS NULL) AS nulos_game_name,
            COUNT(*) FILTER (WHERE platform IS NULL) AS nulos_platform,
            COUNT(*) FILTER (WHERE release_year IS NULL) AS nulos_release_year,
            COUNT(*) FILTER (WHERE genre IS NULL) AS nulos_genre,
            COUNT(*) FILTER (WHERE publisher IS NULL) AS nulos_publisher,
            COUNT(*) FILTER (WHERE na_sales IS NULL) AS nulos_na_sales,
            COUNT(*) FILTER (WHERE eu_sales IS NULL) AS nulos_eu_sales,
            COUNT(*) FILTER (WHERE jp_sales IS NULL) AS nulos_jp_sales,
            COUNT(*) FILTER (WHERE other_sales IS NULL) AS nulos_other_sales,
            COUNT(*) FILTER (WHERE global_sales IS NULL) AS nulos_global_sales
        FROM
            videogame_sales
    """, 
    con=conn
)

,nulos_ranking,nulos_game_name,nulos_platform,nulos_release_year,nulos_genre,nulos_publisher,nulos_na_sales,nulos_eu_sales,nulos_jp_sales,nulos_other_sales,nulos_global_sales
0,0,0,0,271,0,58,0,0,0,0,0


A consulta acima conta todos os elementos presentes em uma coluna em que foi aplicado um filtro que mostra apenas os elementos de valor NULL, repetindo essa mesma contagem e aplicação de filtro para cada uma das colunas.
Assim, temos a contagem de elementos nulos em cada uma das colunas.

#### Valores únicos nas colunas de variáveis categóricas

Temos as seguintes colunas como representantes de variáveis categóricas:
- Plataforma
- Ano de lançamento
- Gênero
- Empresa desenvolvedora

A consulta abaixo pode nos mostrar o número de categorias distintas (valores únicos) em cada uma das colunas

In [91]:
pd.read_sql_query(
    """
        SELECT 
            count(DISTINCT platform) AS unique_platform_count,
            count(DISTINCT release_year) AS unique_release_year_count,
            count(DISTINCT genre) AS unique_genre_count,
            count(DISTINCT publisher) AS unique_publisher_count
        FROM
            videogame_sales
    """, 
    con=conn
)

,unique_platform_count,unique_release_year_count,unique_genre_count,unique_publisher_count
0,31,39,12,578


A consulta acima conta os valores distintos em cada uma das colunas e nos retorna essa contagem.

#### Outliers nas colunas numéricas

Como reperesentantes de variáveis numéricas temos as seguintes colunas:
- na_sales
- eu_sales
- jp_sales
- other_sales
- global_sales

##### Função percentile_cont

Os cálculos a seguir usam a função percentile_cont presente no postgres. Essa função calcula o valor correspondente ao percentil passado como argumento. Para isso, deve ser passado para o WITHIN GROUP o conjunto ordenado de valores que serão utilizados no cálculo ([documentação](https://www.postgresql.org/docs/15/functions-aggregate.html))

##### Definição de outlier

Definiremos como outlier um valor que estiver acima de Q3 + 1.5 * IQR ou abaixo de Q1 - 1.5 * IQR.<br>
Q1 e Q3 são os valores do primeiro e terceiro quartis.<br>
IQR (Intervalo Interquartil) é a diferença Q3 - Q1. 

##### Outliers em na_sales

In [75]:
pd.read_sql_query(
    """
        WITH stats AS (
            SELECT
                PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY na_sales) AS q1,
                PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY na_sales) AS q3
            FROM
                videogame_sales
        )
        SELECT
            COUNT(na_sales)
        FROM
            videogame_sales, stats
        WHERE
            na_sales < stats.q1 - 1.5 * (stats.q3 - stats.q1) OR
            na_sales > stats.q3 + 1.5 * (stats.q3 - stats.q1);
    """, 
    con=conn
)

,count
0,1711


Temos Q1 = 0 e Q3 = 0.24.<br>
Assim, nosso IQR vale 0.36.<br>
São outliers valores acima de 0.6 ou abaixo de -0.36.<br>
Como não temos valores negativos para vendas, não há outliers "para baixo".<br>
Há 1711 outliers "para cima".<br> 
Não há motivo para desconsiderá-los, uma vez que não representam erros de medida ou algo do tipo. São apenas jogos que tiveram vendas excepcionais quando comparados ao resto.

##### Outliers em eu_sales

In [82]:
pd.read_sql_query(
    """
        WITH stats AS (
            SELECT
                PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY eu_sales) AS q1,
                PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY eu_sales) AS q3
            FROM
                videogame_sales
        )
        SELECT
            COUNT(eu_sales)
        FROM
            videogame_sales, stats
        WHERE
            eu_sales < stats.q1 - 1.5 * (stats.q3 - stats.q1) OR
            eu_sales > stats.q3 + 1.5 * (stats.q3 - stats.q1)
    """, 
    con=conn
)

,count
0,2081


Temos Q1 = 0 e Q3 = 0.11.<br>
Assim, nosso IQR é 0.11.<br>
São outliers valores abaixo de -0.165 ou acima de 0.275.<br>
Da mesma forma que em na_sales, não temos outliers "para baixo".<br>
Existem 2081 outliers "para cima".<br>
O tratamento dado aos outliers será o mesmo de na_sales.<br>

##### Outliers em jp_sales

In [90]:
pd.read_sql_query(
    """
        WITH stats AS (
            SELECT
                PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY jp_sales) AS q1,
                PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY jp_sales) AS q3
            FROM
                videogame_sales
        )
        SELECT
            COUNT(jp_sales)
        FROM
            videogame_sales, stats
        WHERE
            jp_sales < stats.q1 - 1.5 * (stats.q3 - stats.q1) OR
            jp_sales > stats.q3 + 1.5 * (stats.q3 - stats.q1)
    """, 
    con=conn
)

,count
0,2577


Temos Q1 = 0 e Q3 = 0.04.<br>
Assim, nosso IQR é 0.04.<br>
Serão outliers valores abaixo de -0.02 ou acima de 0.1.<br>
Não temos outliers "para baixo".<br>
Existem 2577 outliers "para cima".<br>
O tratamento dado aos outliers será o mesmo de antes.<br>

##### Outliers em other_sales

In [11]:
pd.read_sql_query(
    """
        WITH stats AS (
            SELECT
                PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY other_sales) AS q1,
                PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY other_sales) AS q3
            FROM
                videogame_sales
        )
        SELECT
            COUNT(other_sales)
        FROM
            videogame_sales, stats
        WHERE
            other_sales < stats.q1 - 1.5 * (stats.q3 - stats.q1) OR
            other_sales > stats.q3 + 1.5 * (stats.q3 - stats.q1)
    """, 
    con=conn
)

,count
0,1832


Temos Q1 = 0 e Q3 = 0.04.<br>
Assim, nosso IQR é 0.04.<br>
Serão outliers valores abaixo de -0.02 ou acima de 0.1.<br>
Não temos outliers "para baixo".<br>
Existem 1832 outliers "para cima".<br>
O tratamento dado aos outliers será o mesmo de antes.<br>

##### Outliers em global_sales

In [96]:
pd.read_sql_query(
    """
        WITH stats AS (
            SELECT
                PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY global_sales) AS q1,
                PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY global_sales) AS q3
            FROM
                videogame_sales
        )
        SELECT
            COUNT(global_sales)
        FROM
            videogame_sales, stats
        WHERE
            global_sales < stats.q1 - 1.5 * (stats.q3 - stats.q1) OR
            global_sales > stats.q3 + 1.5 * (stats.q3 - stats.q1)
    """, 
    con=conn
)

,count
0,1893


Temos Q1 = 0.06 e Q3 = 0.47.<br>
Assim, nosso IQR é 0.39.<br>
Serão outliers valores abaixo de -0.535 ou acima de 1.055.<br>
Não temos outliers "para baixo".<br>
Existem 1893 outliers "para cima".<br>
O tratamento dado aos outliers será o mesmo de antes.<br>

### Perguntas

#### Pergunta 1 - Qual a soma de todos os jogos vendidos no Wii?

In [12]:
pd.read_sql_query(
    """
        SELECT 
            SUM(global_sales)
        FROM 
            videogame_sales
        WHERE 
            platform = 'Wii'
    """, 
    con=conn
)

,sum
0,926.7112


Essa query filtra a plataforma com nome "Wii" e soma todos os seus global_sales.

#### Pergunta 2 - Quais foram os 5 jogos lançados entre 2010 e 2016 que mais venderam?

In [17]:
pd.read_sql_query(
    """
        SELECT 
            game_name, 
            SUM(global_sales) AS global_sales_sum,
            release_year
        FROM 
            videogame_sales
        WHERE 
            release_year <= 2016 AND release_year >= 2010
        GROUP BY 
            game_name, release_year
        ORDER BY 
            global_sales_sum DESC
        LIMIT 
            5
    """, 
    con=conn
)

,game_name,global_sales_sum,release_year
0,Grand Theft Auto V,37.780000,2013
1,Call of Duty: Modern Warfare 3,30.830000,2011
2,Call of Duty: Black Ops II,29.720000,2012
3,Call of Duty: Black Ops,29.399998,2010
4,Call of Duty: Ghosts,27.380001,2013


Essa query agrupa todos os jogos com mesmo nome e seus anos de lançamento, filtra os jogos entre 2010 e 2016 e soma seus global_sales. Ao final, ordena os jogos por venda e limita a quantidade de jogos mostrados em 5.

#### Pergunta 3 - Qual região foi responsável pela maior quantidade de vendas  de jogos?

In [20]:
pd.read_sql_query(
    """
        SELECT 
            'na_sales' AS region_sales, 
            SUM(na_sales) AS valor
        FROM 
            videogame_sales
        UNION ALL
        SELECT 
            'eu_sales' AS region_sales, 
            SUM(eu_sales) AS valor
        FROM 
            videogame_sales
        UNION ALL
        SELECT 
            'jp_sales' AS region_sales, 
            SUM(jp_sales) AS valor
        FROM 
            videogame_sales
        UNION ALL
        SELECT 
            'other_sales' AS region_sales, 
            SUM(other_sales) AS valor
        FROM 
            videogame_sales
        ORDER BY 
            valor DESC
        LIMIT 
            1
    """, 
    con=conn
)

,region_sales,valor
0,na_sales,4392.5713


Essa query soma as vendas de um região e cria uma string para nomeá-la, em seguida, faz o mesmo com as outras e as une em uma única tabela. Por fim, ordena os valores de venda em ordem decrescente e mostra apenas o primeiro dado.

#### Pergunta 4 - Quais as 3 plataformas com mais jogos vendidos?

In [21]:
pd.read_sql_query(
    """
        SELECT 
            platform, 
            SUM(global_sales) AS sum_global_sales
        FROM 
            videogame_sales
        GROUP BY 
            platform
        ORDER BY 
            sum_global_sales DESC
        LIMIT 
            3
    """, 
    con=conn
)

,platform,sum_global_sales
0,PS2,1255.6489
1,X360,979.9612
2,PS3,957.8420


Essa query agrupa as plataformas de mesmo nome e soma os seus global_sales. Em seguida, ela ordena em ordem decrescente pelo sum_global_sales e limita a visualização em 3.

#### Pergunta 5 - Quais os 10 jogos mais vendidos no japão?

In [22]:
pd.read_sql_query(
    """
        SELECT 
            game_name, 
            SUM(jp_sales) AS sum_jp_sales
        FROM 
            videogame_sales
        GROUP BY 
            game_name
        ORDER BY 
            sum_jp_sales DESC
        LIMIT 
            10
    """, 
    con=conn
)

,game_name,sum_jp_sales
0,Pokemon Red/Pokemon Blue,10.22
1,Pokemon Gold/Pokemon Silver,7.20
2,Super Mario Bros.,6.96
3,New Super Mario Bros.,6.50
4,Pokemon Diamond/Pokemon Pearl,6.04
5,Tetris,6.03
6,Pokemon Black/Pokemon White,5.65
7,Dragon Quest VII: Warriors of Eden,5.40
8,Pokemon Ruby/Pokemon Sapphire,5.38
9,Animal Crossing: Wild World,5.33


Essa query junta os jogos de mesmo nome e soma as suas jp_sales. Em seguida ordena em ordem decrescente pelo sum_jp_sales e limita sua visualização em 10.

#### Pergunta 6 - Quantos jogos foram lançados no ano de 2008?

In [45]:
pd.read_sql_query(
    """
        SELECT 
            COUNT(DISTINCT game_name) AS games_released_in_2008
        FROM 
            videogame_sales
        WHERE 
            release_year = 2008
    """, 
    con=conn
)

,games_released_in_2008
0,1039


Essa query filtra todos os jogos que foram lançados em 2008 e conta sua quantidade.

#### Pergunta 7 - Quais as 3 plataformas com mais jogos lançados?

In [46]:
pd.read_sql_query(
    """
        SELECT 
            platform, 
            COUNT(game_name) AS count_game_name
        FROM 
            videogame_sales
        GROUP BY 
            platform
        ORDER BY 
            count_game_name DESC
        LIMIT 
            3
    """, 
    con=conn
)

,platform,count_game_name
0,DS,2163
1,PS2,2161
2,PS3,1329


Essa query agrupa as plataformas de mesmo nome e conta a quantidade de jogos em cada uma. Em seguida ordena de forma decrescente pela quantidade de jogos e limita sua visualização em 3.

#### Pergunta 8 - A soma das vendas dos 3 jogos lançados em maior número de plataformas?

In [4]:
pd.read_sql_query(
    """
        SELECT 
            game_name, 
            COUNT(platform) AS count_platform, 
            SUM(global_sales) AS sum_global_sales
        FROM 
            videogame_sales
        GROUP BY 
            game_name
        ORDER BY 
            count_platform DESC
        LIMIT 
            3
    """, 
    con=conn
)

,game_name,count_platform,sum_global_sales
0,Need for Speed: Most Wanted,12,14.080001
1,Madden NFL 07,9,10.030001
2,FIFA 14,9,17.140001


Essa query agrupa os jogos de mesmo nome, conta a quantidade de plataformas de cada jogo e soma seu global_sales. Em seguida, ordena em ordem decrescente pelo count_platform e limita sua visualização em 3.

#### Pergunta 9 - Quantos jogos de cada gênero cada desenvolvedora lançou?

Para rodar a query que responde essa consulta vamos habilitar uma extensão no postgres chamada tablefunc.<br>
Para habilitar essa extensão, rode a célula abaixo (só precisa habilitar uma vez, por isso deixamos em uma célula separada.)<br>
Essa extensão possui uma função chamada [CROSSTAB](https://www.postgresql.org/docs/current/tablefunc.html#TABLEFUNC-FUNCTIONS) que nos permite pivotar uma tabela.<br>

In [ ]:
pd.read_sql_query(
    """
        CREATE EXTENSION tablefunc;
    """, 
    con=conn
)

In [10]:
pd.read_sql_query(
    """
        SELECT * FROM CROSSTAB(
            'SELECT 
                publisher,
                genre,
                COUNT(genre) AS count_genre
            FROM
                videogame_sales
            GROUP BY 
                genre, 
                publisher
            ORDER BY 
                publisher, 
                count_genre DESC',
            'SELECT DISTINCT
                genre
            FROM
                videogame_sales'
        ) AS genre_by_publisher (
            publisher varchar(50),
            adventure int,
            puzzle int,
            strategy int,
            rpg int,
            simulation int,
            misc int,
            fighting int,
            racing int,
            sports int,
            action int,
            shooter int,
            platform int
        )
    """, 
    con=conn
)

,publisher,adventure,puzzle,strategy,rpg,simulation,misc,fighting,racing,sports,action,shooter,platform
0,10TACLE Studios,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1C Company,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,20th Century Fox Video Games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,NaN
3,2D Boy,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3DO,3.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,6.0,17.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,Zenrin,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
575,Zoo Digital Publishing,7.0,14.0,1.0,NaN,1.0,18.0,3.0,26.0,16.0,8.0,8.0,2.0
576,Zoo Games,1.0,3.0,NaN,NaN,1.0,11.0,NaN,6.0,4.0,3.0,4.0,NaN
577,Zushi Games,1.0,2.0,2.0,1.0,2.0,1.0,NaN,3.0,3.0,1.0,2.0,NaN


Essa query funciona da seguinte maneira:
- Faz duas queries base
    - Query para definir a tabela base contendo publishers, gêneros de jogo e quantos jogos daquele gênero cada publisher desenvolveu
    - Query para definir uma tabela cujas linhas serão as colunas da nossa tabela final contendo todos os nomes de gênero presentes na tabela
- Essas duas queries são passadas como argumentos para a função crosstab, que se encarregará de fazer o pivoteamento
- Em seguida, precisamos definir os nomes e tipos das colunas da tabela final
    - Segundo a [documentação](https://www.postgresql.org/docs/current/tablefunc.html#TABLEFUNC-FUNCTIONS), se os dados retornados fossem todos do tipo texto, poderíamos evitar essa definição usando a função crosstabN. No entanto, não conseguimos fazẽ-la funcionar. Mesmo fazendo o cast da contagem para text, o retorno do erro nos mostra que as funções crosstab2/3/4 (segundo a documentação, são as que existem) não foram encontradas.


#### Pergunta 10 - Quais os nomes, anos de lançamento, gêneros e desenvolvedoras de cada jogo que mais vendeu em cada uma das regiões?  

In [6]:
pd.read_sql_query(
    """
        WITH RankedGames AS (
            SELECT
                game_name,
                genre,
                publisher,
                release_year,
                ROW_NUMBER() OVER (ORDER BY na_sales DESC) AS na_rank,
                ROW_NUMBER() OVER (ORDER BY eu_sales DESC) AS eu_rank,
                ROW_NUMBER() OVER (ORDER BY jp_sales DESC) AS jp_rank,
                ROW_NUMBER() OVER (ORDER BY other_sales DESC) AS other_rank,
                ROW_NUMBER() OVER (ORDER BY global_sales DESC) AS global_rank
            FROM
                videogame_sales
        )
        SELECT
            'North America' AS best_in,
            game_name,
            release_year,
            genre,
            publisher
        FROM
            RankedGames
        WHERE
            na_rank = 1
        UNION ALL
        SELECT
            'Europe' AS best_in,
            game_name,
            release_year,
            genre,
            publisher
        FROM
            RankedGames
        WHERE
            eu_rank = 1
        UNION ALL
        SELECT
            'Japan' AS best_in,
            game_name,
            release_year,
            genre,
            publisher
        FROM
            RankedGames
        WHERE
            jp_rank = 1
        UNION ALL
        SELECT
            'Other Regions' AS best_in,
            game_name,
            release_year,
            genre,
            publisher
        FROM
            RankedGames
        WHERE
            other_rank = 1
        UNION ALL
        SELECT
            'Global' AS best_in,
            game_name,
            release_year,
            genre,
            publisher
        FROM
            RankedGames
        WHERE
            global_rank = 1;
    """, 
    con=conn
)

,best_in,game_name,release_year,genre,publisher
0,North America,Wii Sports,2006,Sports,Nintendo
1,Europe,Wii Sports,2006,Sports,Nintendo
2,Japan,Pokemon Red/Pokemon Blue,1996,Role-Playing,Nintendo
3,Other Regions,Grand Theft Auto: San Andreas,2004,Action,Take-Two Interactive
4,Global,Wii Sports,2006,Sports,Nintendo


Começamos a query criando uma tabela auxiliar chamada RankedGames que mostra os dados que queremos em cada um dos jogos e cria colunas que mostram qual a posição do jogo (usando a função [ROW_NUMBER](https://www.postgresql.org/docs/current/functions-window.html)) quando ordenamos videogame_sales usando cada uma das colunas de vendas (na_sales, eu_sales, jp_sales, other_sales, global_sales).<br>
Depois, fazemos 5 consultas nessa tabela auxiliar para pegar cada um dos jogos que estão na posição 1 em cada uma dessas ordenações, criamos uma coluna best_in para dizer em qual região esse jogo foi o melhor e "colamos" essas tabelas de 1 linha umas nas outras usando UNION ALL. 